## Lab 6-1: Softmax

In [2]:
import tensorflow as tf

In [3]:
tf.set_random_seed(777)  # for reproducibility

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [4]:
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3]) # 3 = number of classes
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [5]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
            
    a = sess.run(hypothesis, feed_dict = {X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.arg_max(a, 1)))
    
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9],
                                              [1, 3, 4, 3],
                                              [1, 1, 0, 1]]})
    print(all, sess.run(tf.arg_max(all, 1)))

0 5.04862
200 0.548425
400 0.432149
600 0.352467
800 0.276829
1000 0.232683
1200 0.210553
1400 0.192215
1600 0.176752
1800 0.163534
2000 0.152108
[[  1.38904958e-03   9.98601854e-01   9.06129117e-06]] [1]
[[  1.38904958e-03   9.98601854e-01   9.06129117e-06]
 [  9.31192040e-01   6.29020557e-02   5.90589503e-03]
 [  1.27327668e-08   3.34112905e-04   9.99665856e-01]] [1 0 2]


## Softmax cross entropy with logits

In [10]:
import numpy as np

In [11]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [12]:
nb_classes = 7 # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [13]:
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                labels=Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

In [15]:
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict ={
                                        X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))
            
    pred = sess.run(prediction, feed_dict = {X: x_data})
    for p, y in zip(pred, y_data.flatten()):
        print(p==int(y), p, int(y))

Step:     0	Loss: 7.486	Acc: 1.98%
Step:   100	Loss: 0.698	Acc: 71.29%
Step:   200	Loss: 0.400	Acc: 90.10%
Step:   300	Loss: 0.281	Acc: 96.04%
Step:   400	Loss: 0.215	Acc: 96.04%
Step:   500	Loss: 0.175	Acc: 99.01%
Step:   600	Loss: 0.147	Acc: 100.00%
Step:   700	Loss: 0.128	Acc: 100.00%
Step:   800	Loss: 0.113	Acc: 100.00%
Step:   900	Loss: 0.101	Acc: 100.00%
Step:  1000	Loss: 0.092	Acc: 100.00%
Step:  1100	Loss: 0.084	Acc: 100.00%
Step:  1200	Loss: 0.078	Acc: 100.00%
Step:  1300	Loss: 0.073	Acc: 100.00%
Step:  1400	Loss: 0.068	Acc: 100.00%
Step:  1500	Loss: 0.064	Acc: 100.00%
Step:  1600	Loss: 0.060	Acc: 100.00%
Step:  1700	Loss: 0.057	Acc: 100.00%
Step:  1800	Loss: 0.054	Acc: 100.00%
Step:  1900	Loss: 0.051	Acc: 100.00%
True 0 0
True 0 0
True 3 3
True 0 0
True 0 0
True 0 0
True 0 0
True 3 3
True 3 3
True 0 0
True 0 0
True 1 1
True 3 3
True 6 6
True 6 6
True 6 6
True 1 1
True 0 0
True 3 3
True 0 0
True 1 1
True 1 1
True 0 0
True 1 1
True 5 5
True 4 4
True 4 4
True 0 0
True 0 0
True 0